In [1]:
import pandas as pd
import json 

### Defining rules

In [2]:
#DEFINITIONS
POSTAL_CODE = "codigo_postal"
SUBURB = "colonia"
STATE = "estado"
MUNICIPALITY = "municipio"
CITY = "ciudad"
INPUT_FOLDER = "original_database/"
INPUT_FILE = "CPdescarga.xls"
OUTPUT_FOLDER = "processed_database/"
JSON_FOLDER = "JSON/"
CSV_FOLDER = "CSV/"
OUTPUT_CSV_FULL = "codigos_postales.csv"
OUTPUT_CSV_REDUCED = "codigos_postaless_reduced.csv"
OUTPUT_CSV_REDUCED_STATE = f"{STATE}.csv"
OUTPUT_CSV_REDUCED_MUNICIPALITY = f"{MUNICIPALITY}.csv"
OUTPUT_CSV_REDUCED_CITY = f"{CITY}.csv"
OUTPUT_JSON_FULL = "codigos_postales.json"
OUTPUT_JSON_REDUCED = "codigos_postales_reduced.json"
MAIN_JSON_KEY = "codigos_postales"

#Used for renaming states
best_known_names_for_states = {"Coahuila de Zaragoza":"Coahuila", "Michoacán de Ocampo":"Michoacán",
                              "Veracruz de Ignacio de la Llave": "Veracruz"}
#Used to Ignore sheets
sheet_names_ignore = ["Nota"]

#Used to include and rename a column
renaming_columns = {"d_codigo":POSTAL_CODE,"d_asenta":SUBURB,"d_estado":STATE,"D_mnpio":MUNICIPALITY,
                    "d_ciudad":CITY}

### Reading source "database"

In [3]:
xls = pd.ExcelFile(f"{INPUT_FOLDER}{INPUT_FILE}")

### Getting States from sheets

In [4]:
states = list(filter(lambda x: x not in sheet_names_ignore, xls.sheet_names))

# Creating CSV

### Merging information from all states in one dataframe

In [5]:
df =  pd.DataFrame(columns = list(renaming_columns.values()))
for state in states:
    df = pd.concat([df,xls.parse(state).rename(columns=renaming_columns)],ignore_index=True)

### Filtering selected columns 

In [6]:
df = df[list(renaming_columns.values())]

### Filling empty values and updating states names

In [7]:
df.fillna("", inplace=True)
df[STATE].replace(best_known_names_for_states, inplace=True)
df.sort_values([POSTAL_CODE,SUBURB],inplace=True)

### Creating normalized CSV

In [8]:
df_norm = df.copy()
keyDict = {STATE:[],MUNICIPALITY:[],CITY:[]}
df_norm[STATE], keyDict[STATE] = pd.factorize(df[STATE])
df_norm[MUNICIPALITY], keyDict[MUNICIPALITY] = pd.factorize(df[MUNICIPALITY])
df_norm[CITY], keyDict[CITY] = pd.factorize(df[CITY])
df_STATE = pd.DataFrame(data =keyDict[STATE],  columns = [STATE])
df_MUNICIPALITY = pd.DataFrame(data =keyDict[MUNICIPALITY],  columns = [MUNICIPALITY])
df_CITY = pd.DataFrame(data =keyDict[CITY],  columns = [CITY])

### Writing CSV Files

In [13]:
df.to_csv(f"{OUTPUT_FOLDER}{CSV_FOLDER}{OUTPUT_CSV_FULL}",index=False,encoding='utf-8')
df_norm.to_csv(f"{OUTPUT_FOLDER}{CSV_FOLDER}{OUTPUT_CSV_REDUCED}",index=False,encoding='utf-8')
df_STATE.to_csv(f"{OUTPUT_FOLDER}{CSV_FOLDER}{OUTPUT_CSV_REDUCED_STATE}",encoding='utf-8')
df_MUNICIPALITY.to_csv(f"{OUTPUT_FOLDER}{CSV_FOLDER}{OUTPUT_CSV_REDUCED_MUNICIPALITY}",encoding='utf-8')
df_CITY.to_csv(f"{OUTPUT_FOLDER}{CSV_FOLDER}{OUTPUT_CSV_REDUCED_CITY}",encoding='utf-8')


# Creating JSON

### Creating JSON Structures

In [10]:
full_json_data = {MAIN_JSON_KEY:{}}
reduced_json_data = {MAIN_JSON_KEY:{}}
postal_codes = df[POSTAL_CODE].unique()
for postal_code in postal_codes:
    postal_code = int(postal_code)
    rows = df[df[POSTAL_CODE]==postal_code]
    #print(rows.keys())
    #full_json_data[postal_code] = {STATE:list(rows[STATE]), MUNICIPALITY:list(rows[MUNICIPALITY]), 
    #                               CITY:list(rows[CITY]), SUBURB:list(rows[SUBURB])}
    for row in rows.iloc():
        new_reg = {SUBURB:row[SUBURB], STATE:row[STATE],MUNICIPALITY:row[MUNICIPALITY], CITY:row[CITY]}
        if(postal_code in full_json_data[MAIN_JSON_KEY]):
            full_json_data[MAIN_JSON_KEY][postal_code].append(new_reg)
        else:
            full_json_data[MAIN_JSON_KEY][postal_code] = [new_reg]
    reduced_json_data[MAIN_JSON_KEY][postal_code] = {STATE:list(rows[STATE])[0], MUNICIPALITY:list(rows[MUNICIPALITY])[0], 
                                   CITY:list(rows[CITY])[0], SUBURB:list(dict.fromkeys(list(rows[SUBURB])))}
       

### Writing JSON Files

In [11]:
with open(f"{OUTPUT_FOLDER}{JSON_FOLDER}{OUTPUT_JSON_FULL}", 'w+', encoding='utf-8') as nf:
    data = json.dumps(full_json_data, ensure_ascii=False, indent=4)
    nf.write(data)

with open(f"{OUTPUT_FOLDER}{JSON_FOLDER}{OUTPUT_JSON_REDUCED}", 'w+', encoding='utf-8') as nf:
    data = json.dumps(reduced_json_data, ensure_ascii=False, indent=4)
    nf.write(data)